# How to fit Parametric Forecasts to Quantile Forecasts

In [1]:
from create_parametric_forecasts import ParametricForecasts
import pandas as pd
import numpy as np

In [2]:
#path_to_quantile_fc = 'C:\\Users\\fh6281\\PycharmProjects\\GermanBuildingDate\\02_forecast\\storage_quantile_fc\\file_fc_PatchTST_SFH3_2025-07-16_13-47-24_freq15.csv'
#path_to_quantile_fc = 'C:\\Users\\fh6281\\PycharmProjects\\GermanBuildingDate\\02_forecast\\storage_quantile_fc\\file_fc_AutoKAN_SFH3_2025-07-25_14-18-05_freq15.csv'
path_to_quantile_fc = 'C:\\Users\\fh6281\\PycharmProjects\\GermanBuildingDate\\02_forecast\\storage_quantile_fc\\file_fc_AutoKAN_SFH3_short.csv'

pf = ParametricForecasts()
pf.load_quantile_forecasts(path_to_quantile_fc, timerange=['2020-05-13 06:15:00+00:00', '2020-05-14 06:45:00+00:00'])  # timerange=None

In [3]:
pf.sort_quantiles()
pf.quantile_forecasts

Sorting quantiles in ascending order…


0.01       0.02  \
time_fc_created           timestamp                                         
2020-05-13 06:15:00+00:00 2020-05-13 06:15:00+00:00 -2684.0806 -2380.6167   
                          2020-05-13 06:30:00+00:00 -2890.3403 -2645.7670   
                          2020-05-13 06:45:00+00:00 -3379.8320 -3073.0806   
                          2020-05-13 07:00:00+00:00 -3798.5127 -3586.4620   
                          2020-05-13 07:15:00+00:00 -4161.5170 -3856.0190   
...                                                        ...        ...   
2020-05-14 06:30:00+00:00 2020-05-15 06:15:00+00:00 -3693.1064 -3305.7110   
2020-05-14 06:45:00+00:00 2020-05-14 06:45:00+00:00 -4716.2373 -4362.6170   
                          2020-05-14 07:00:00+00:00 -5219.6475 -4633.0460   
                          2020-05-14 07:15:00+00:00 -5716.6455 -5283.3680   
                          2020-05-14 07:30:00+00:00 -6187.6104 -5674.0390   

                                                          0.03       0.04  \
time_fc_created           timestamp                                         
2020-05-13 06:15:00+00:00 2020-05-13 06:15:00+00:00 -2115.7070 -1970.4945   
                          2020-05-13 06:30:00+00:00 -2276.6685 -2268.3423   
                          2020-05-13 06:45:00+00:00 -2675.7036 -2596.2617   
                          2020-05-13 07:00:00+00:00 -3106.3047 -3077.5063   
                          2020-05-13 07:15:00+00:00 -3512.1553 -3324.5170   
...                                                        ...        ...   
2020-05-14 06:30:00+00:00 2020-05-15 06:15:00+00:00 -3083.8882 -2918.4238   
2020-05-14 06:45:00+00:00 2020-05-14 06:45:00+00:00 -3993.8210 -3737.0990   
                          2020-05-14 07:00:00+00:00 -4318.5890 -4134.5770   
                          2020-05-14 07:15:00+00:00 -4700.8633 -4520.2460   
                          2020-05-14 07:30:00+00:00 -5282.0030 -5195.3670   

                                                          0.05       0.06  \
time_fc_created           timestamp                                         
2020-05-13 06:15:00+00:00 2020-05-13 06:15:00+00:00 -1850.6685 -1785.6593   
                          2020-05-13 06:30:00+00:00 -2225.6104 -2129.3384   
                          2020-05-13 06:45:00+00:00 -2424.3540 -2416.5903   
                          2020-05-13 07:00:00+00:00 -2880.1792 -2829.5820   
                          2020-05-13 07:15:00+00:00 -3149.9834 -3061.5996   
...                                                        ...        ...   
2020-05-14 06:30:00+00:00 2020-05-15 06:15:00+00:00 -2740.6426 -2672.7466   
2020-05-14 06:45:00+00:00 2020-05-14 06:45:00+00:00 -3675.2710 -3566.7932   
                          2020-05-14 07:00:00+00:00 -4093.5042 -3977.7593   
                          2020-05-14 07:15:00+00:00 -4361.0693 -4358.2715   
                          2020-05-14 07:30:00+00:00 -4870.1777 -4818.8076   

                                                          0.07       0.08  \
time_fc_created           timestamp                                         
2020-05-13 06:15:00+00:00 2020-05-13 06:15:00+00:00 -1753.3043 -1609.5797   
                          2020-05-13 06:30:00+00:00 -2065.7014 -2003.8525   
                          2020-05-13 06:45:00+00:00 -2366.4646 -2311.3398   
                          2020-05-13 07:00:00+00:00 -2814.6567 -2690.3096   
                          2020-05-13 07:15:00+00:00 -2986.1826 -2969.2050   
...                                                        ...        ...   
2020-05-14 06:30:00+00:00 2020-05-15 06:15:00+00:00 -2645.2266 -2613.2822   
2020-05-14 06:45:00+00:00 2020-05-14 06:45:00+00:00 -3456.2856 -3413.4712   
                          2020-05-14 07:00:00+00:00 -3875.9443 -3864.9314   
                          2020-05-14 07:15:00+00:00 -4259.1143 -4125.6580   
                          2020-05-14 07:30:00+00:00 -4661.1035 -4619.6265   

                                                  

In [4]:
pf.fit_distribution('sum2gaussian')

Progress: 100% (9412/9412)

In [5]:
pf.store_parametric_forecasts()  # creation_time=True

Parametric forecasts saved to C:\Users\fh6281\PycharmProjects\GermanBuildingDate\02_forecast\storage_param_fc\file_fc_parametric_AutoKAN_SFH3_short_1.csv


# Playground
